In [3]:
from qdtuner import SingleQuantumDotTuner

In [ ]:
QDTuner = SingleQuantumDotTuner(
    device_config='../device_config.yml',
    station_config='../station_config.yml',
    tuner_config='../tuner_config.yml'
)

In [ ]:
QDTuner.bias_device(Vbias=0.1)

In [ ]:
# Set maxV equal to None if you want to sweep 
# up to the max voltage you set in device config
# It'll stop if it hits max current (it should)
QDTuner.check_turn_on(minV=0, maxV=None, dV=0.005)

In [ ]:
# Set maxV equal to None if you want to use
# saturation voltage found in turn on phase.
QDTuner.check_pinch_offs(minV=1, maxV=None)